|<h1 align="center">Name</h1>|<h1 align="center">ID</h1>|
|---|---|
|<h4 align="center">Abd Allah Mohamed Abd Allah Mohamed Taman</h4>|<h4 align="center">20010906</h4>|
|<h4 align="center">Karim Fathy Abd Alaziz Mohamed Mostafa</h4>|<h4 align="center">20011116</h4>|
|<h4 align="center">Mahmoud Ali Ahmed Ali</h4>|<h4 align="center">20011811</h4>|:


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# <font color='red'>Download Dataset and Understand the Format</font>


In [ ]:
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
import numpy as np
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.decomposition import PCA
import pandas as pd
from scipy.sparse.csgraph import connected_components
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.cluster import contingency_matrix


In [ ]:
def precision_cal(cluster_labels, true_labels):
    unique_labels = np.unique(cluster_labels)
    precision_scores = []
    weights = []
    for label in unique_labels:
        cluster_indices = np.where(cluster_labels == label)[0]
        ground_truth_cluster_labels = true_labels[cluster_indices]
        # Majority class in the cluster
        majority_class = np.argmax(np.bincount(ground_truth_cluster_labels))
        # Predicted labels for the current cluster
        predicted_labels = np.full_like(ground_truth_cluster_labels, majority_class)
        # Compute precision for the current cluster
        precision = precision_score(ground_truth_cluster_labels, predicted_labels, average='weighted', zero_division=1)
        precision_scores.append(precision)
    return np.sum(precision_scores)/len(precision_scores)

def conditional_entropy_cal(cluster_labels, true_labels):
    contingency = contingency_matrix(cluster_labels, true_labels)
    contingency = np.asarray(contingency, dtype=np.float64)
    contingency_sum = np.sum(contingency)
    contingency /= contingency_sum  # convert contingency to probabilities
    true_distribution = np.sum(contingency, axis=1)
    cluster_distribution = np.sum(contingency, axis=0)
    H_c_given_t = -np.sum(contingency * np.log(contingency.clip(min=1e-15)) / np.log(2))
    H_t = -np.sum(true_distribution * np.log(true_distribution.clip(min=1e-15)) / np.log(2))

    return H_c_given_t / H_t

def recall_cal(cluster_labels, true_labels):
    unique_labels = np.unique(cluster_labels)
    recall_scores = []
    for label in unique_labels:
        cluster_indices = np.where(cluster_labels == label)[0]
        ground_truth_cluster_labels = true_labels[cluster_indices]
        majority_class = np.argmax(np.bincount(ground_truth_cluster_labels))
        predicted_labels = np.full_like(ground_truth_cluster_labels, majority_class)
        recall = recall_score(ground_truth_cluster_labels, predicted_labels, average='weighted')
        recall_scores.append(recall)
    return np.average(recall_scores)

def f1_cal(cluster_labels, true_labels):
    unique_labels = np.unique(cluster_labels)
    f1_scores = []
    for label in unique_labels:
        cluster_indices = np.where(cluster_labels == label)[0]
        ground_truth_cluster_labels = true_labels[cluster_indices]
        majority_class = np.argmax(np.bincount(ground_truth_cluster_labels))
        predicted_labels = np.full_like(ground_truth_cluster_labels, majority_class)
        f1 = f1_score(ground_truth_cluster_labels, predicted_labels, average='weighted') # Change average to 'weighted'
        f1_scores.append(f1)
    return np.mean(f1_scores)

def evaluate(true_labels, cluster_labels):
    precision = precision_cal(cluster_labels, true_labels)
    recall = recall_cal(cluster_labels, true_labels)
    f1 = f1_cal(cluster_labels, true_labels)
    conditional_entropy = conditional_entropy_cal(cluster_labels, true_labels)
    return (precision, recall, f1, conditional_entropy)

# <font color='red'>K-Means Algorithm => O(knd)</font>

In [ ]:
def kMeans_algo(data, k, epsilon, iterations):
    n_rows = data.shape[0] #no of samples
    random_idx = np.random.RandomState(42).permutation(n_rows)
    centroids = data[random_idx[:k]] # generate random centroids
    clusters = np.zeros(n_rows) # k clusters with data points in each cluster
    proximityMatrix = np.zeros((n_rows, k)) # distance matrix from each sample to each centroid of k centroids to determine the nearest centroid for each data point
    delta = np.inf # is the threshold we initiate it with infinity till it reaches to value <= epsilon
    iteration = 0 # counter to count number of iterations
    while delta > epsilon and iteration < iterations: # algorithm runs till max iterations reached or the total update in centoids <= epsilon
        for i in range(k): # calculate distances from each data point to the k centroids
            proximityMatrix[:, i] = np.linalg.norm(data - centroids[i], axis=1)

        clusters = np.argmin(proximityMatrix, axis=1) # cluster assignment

        iteration += 1
        old_centroids = deepcopy(centroids) # store old centroids before update

        for i in range(k): # centroids update
            centroids[i, :] = np.mean(data[clusters == i, :], axis=0)
            if np.isnan(centroids).any():
                centroids[i] = old_centroids[i]

        delta = np.linalg.norm(centroids - old_centroids) # update delta

    return clusters, centroids # return the k clusters and centroids

# this function is to predict cluster of the new data points given the training centroids and the data points
def predict(centroids, data):
    k = len(centroids)
    n = data.shape[0]
    proximityMatrix = np.zeros((n, k))
    for i in range(k):
        proximityMatrix[:, i] = np.linalg.norm(data - centroids[i], axis=1)
    predicted_labels = np.argmin(proximityMatrix, axis=1)
    return predicted_labels

# <font color='red'>K-Means evaluation.</font>
<h3 style='color:red;'>In this section we show how kmeans algorithm behaves with <br>
1- taking the mean of each column in each segment resulting in 45 features
for each data point.<br>
2- Flattening all the features together in 45 x 125 = 5625 features for each
data point. For this solution, you are required to reduce the number of
dimensions and work on the projected data after reducing the dimensions.</h3>


In [ ]:
def driver_code_kmeans(address):
    D_train, D_test, ground_truth_train, ground_truth_test = read_data(address)
    pca_dim = 45
    for avg_or_pca in range(0, 2, 1):
        print('solution: ', avg_or_pca + 1)
        for K_clusters in [8, 13, 19, 28, 38]:
            if avg_or_pca == 0:
                D_training = reduce_dimensions(D_train, avg_or_pca, pca_dim)
                clusters, centroids = kMeans_algo(np.array(D_training), K_clusters, 0.001, 200)
                train_pred = predict(centroids, np.array(D_training))
                kmeans_precision, kmeans_recall, kmeans_f1, kmeans_conditional_entropy = evaluate(
                    ground_truth_train,
                    train_pred)
                print('train on k = ', K_clusters)
                print(
                    f'precision : {kmeans_precision}, recall: {kmeans_recall}, f1-measure: {kmeans_f1}, cond_entropy: {kmeans_conditional_entropy}, ')
                D_testing = reduce_dimensions(D_test, avg_or_pca, pca_dim)
                test_pred = predict(centroids, np.array(D_testing))
                kmeans_precision, kmeans_recall, kmeans_f1, kmeans_conditional_entropy = evaluate(
                    ground_truth_test,
                    test_pred)
                print('test on k = ', K_clusters)
                print(
                    f'precision : {kmeans_precision}, recall: {kmeans_recall}, f1-measure: {kmeans_f1}, cond_entropy: {kmeans_conditional_entropy}, ')
            else:
                for pca_dim in [45, 200, 600, 800, 100]:
                    D_training = reduce_dimensions(D_train, avg_or_pca, pca_dim)
                    clusters, centroids = kMeans_algo(np.array(D_training), K_clusters, 0.001, 200)
                    train_pred = predict(centroids, np.array(D_training))
                    kmeans_precision, kmeans_recall, kmeans_f1, kmeans_conditional_entropy = evaluate(
                        ground_truth_train,
                        train_pred)
                    print('train on k = ', K_clusters)
                    print(f'dimensions: {pca_dim} ' +
                          f'precision : {kmeans_precision}, recall: {kmeans_recall}, f1-measure: {kmeans_f1}, cond_entropy: {kmeans_conditional_entropy}, ')
                    D_testing = reduce_dimensions(D_test, avg_or_pca, pca_dim)
                    test_pred = predict(centroids, np.array(D_testing))
                    kmeans_precision, kmeans_recall, kmeans_f1, kmeans_conditional_entropy = evaluate(
                        ground_truth_test,
                        test_pred)
                    print('test on k = ', K_clusters)
                    print(f'dimensions: {pca_dim} ' +
                          f'precision : {kmeans_precision}, recall: {kmeans_recall}, f1-measure: {kmeans_f1}, cond_entropy: {kmeans_conditional_entropy}, ')


driver_code_kmeans(path)

# <font color='lightgreen'>Some notes according to kmeans evaluation.</font>
### <li>the results in the second solution is better than first one as in the first one we just take the mean of each column which summation of the features numbers may result in eliminating each other and don't add a value especially if the feature values is different and has a good variance</li>
### <li>Second solution helps us to add a valuable feature values and thus the clustering is better.</li>
### <li>in case of the second solution we have observed from the results of the evaluation that if we increase the dimensions taken in consideration, then we will obtain better results but this in majority but not in all cases, As there are cases we increased the dimensions taken for the data point but with results getting worse, this is because it may cause noise to the data more than it adds.</li>

# <font color='lightgreen'>Best K in the 1st solution.</font>
### From our analysis, we can say that with k = 28 we get best results in test set clustering.

# <font color='lightgreen'>Best K and dimensions in 2nd solution.</font>
### k = 8 and #dims = 600.

<font color='violet'>
<table>
  <thead>
    <tr>
      <th colspan="2" style="color: violet;"><b>solution: 1</b></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>train on k = 8</td>
      <td>
        <table>
          <thead>
            <tr>
              <th>Metric</th>
              <th>Value</th>
            </tr>
          </thead>
          <tbody>
            <tr>
              <td>Precision</td>
              <td>0.327988</td>
            </tr>
            <tr>
              <td>Recall</td>
              <td>0.879735</td>
            </tr>
            <tr>
              <td>F1-Score</td>
              <td>0.543788</td>
            </tr>
            <tr>
              <td>Conditional Entropy</td>
              <td>1.703264</td>
            </tr>
          </tbody>
        </table>
      </td>
    </tr>
    <!-- More rows for other metrics and values -->
  </tbody>
  <thead>
    <tr>
      <th colspan="2" style="color: violet;"><b>solution: 1</b></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>train on k = 8</td>
      <td>
        <table>
          <thead>
            <tr>
              <th>Metric</th>
              <th>Value</th>
            </tr>
          </thead>
          <tbody>
            <tr>
              <td>Precision</td>
              <td>0.327988</td>
            </tr>
            <tr>
              <td>Recall</td>
              <td>0.879735</td>
            </tr>
            <tr>
              <td>F1-Score</td>
              <td>0.543788</td>
            </tr>
            <tr>
              <td>Conditional Entropy</td>
              <td>1.703264</td>
            </tr>
          </tbody>
        </table>
      </td>
    </tr>
    <!-- More rows for other metrics and values -->
  </tbody>
  <thead>
    <tr>
      <th colspan="2" style="color: violet;"><b>solution: 1</b></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>train on k = 8</td>
      <td>
        <table>
          <thead>
            <tr>
              <th>Metric</th>
              <th>Value</th>
            </tr>
          </thead>
          <tbody>
            <tr>
              <td>Precision</td>
              <td>0.327988</td>
            </tr>
            <tr>
              <td>Recall</td>
              <td>0.879735</td>
            </tr>
            <tr>
              <td>F1-Score</td>
              <td>0.543788</td>
            </tr>
            <tr>
              <td>Conditional Entropy</td>
              <td>1.703264</td>
            </tr>
          </tbody>
        </table>
      </td>
    </tr>
    <!-- More rows for other metrics and values -->
  </tbody>
  <thead>
    <tr>
      <th colspan="2" style="color: violet;"><b>solution: 1</b></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>train on k = 8</td>
      <td>
        <table>
          <thead>
            <tr>
              <th>Metric</th>
              <th>Value</th>
            </tr>
          </thead>
          <tbody>
            <tr>
              <td>Precision</td>
              <td>0.327988</td>
            </tr>
            <tr>
              <td>Recall</td>
              <td>0.879735</td>
            </tr>
            <tr>
              <td>F1-Score</td>
              <td>0.543788</td>
            </tr>
            <tr>
              <td>Conditional Entropy</td>
              <td>1.703264</td>
            </tr>
          </tbody>
        </table>
      </td>
    </tr>
    <!-- More rows for other metrics and values -->
  </tbody>
  <thead>
    <tr>
      <th colspan="2" style="color: violet;"><b>solution: 1</b></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>train on k = 8</td>
      <td>
        <table>
          <thead>
            <tr>
              <th>Metric</th>
              <th>Value</th>
            </tr>
          </thead>
          <tbody>
            <tr>
              <td>Precision</td>
              <td>0.327988</td>
            </tr>
            <tr>
              <td>Recall</td>
              <td>0.879735</td>
            </tr>
            <tr>
              <td>F1-Score</td>
              <td>0.543788</td>
            </tr>
            <tr>
              <td>Conditional Entropy</td>
              <td>1.703264</td>
            </tr>
          </tbody>
        </table>
      </td>
    </tr>
    <!-- More rows for other metrics and values -->
  </tbody>
</table>
</font>

In [ ]:
def read_data(address):
    D_train = []
    D_test = []
    ground_truth_train = []
    ground_truth_test = []
    for activity in range(1, 20):
        for person in range(1, 9):
            for segment in range(1, 61):
                a = str(activity)
                p = str(person)
                s = str(segment)
                if(activity < 10):
                    a = "0" + a
                if(segment < 10):
                    s = "0" + s
                p = f'a{a}/p{p}/s{s}.txt'
                data_point = pd.read_csv(address+p, header=None)
                if(segment < 49):
                    D_train.append(data_point)
                    ground_truth_train.append(activity)
                else:
                    D_test.append(data_point)
                    ground_truth_test.append(activity)

    D_train = np.array(D_train)
    ground_truth_train = np.array(ground_truth_train)
    D_test = np.array(D_test)
    ground_truth_test = np.array(ground_truth_test)
    return D_train, D_test, ground_truth_train, ground_truth_test
def average_on_columns(D):
    new_D = []
    for i in range(len(D)):
        avg_data_point = np.mean(D[i], axis=0)
        new_D.append(avg_data_point)
    return new_D
def flatten(D):
    new_D = []
    for i in range(len(D)):
        L = []
        for j in range(len(D[i])):
            for k in range(len(D[i][j])):
                L.append(D[i][j][k])
        new_D.append(L)
    return new_D
def pca_performer(D, pca_dim):
    pca = PCA(n_components = pca_dim)
    return pca.fit_transform(flatten(D))
def reduce_dimensions(D, avg_or_pca, pca_dim):
    if avg_or_pca == 0:
        return average_on_columns(D)
    else:
        return pca_performer(D, pca_dim)

In [ ]:
def compute_rbf(affinity_matrix, q, gamma):
    rbf_sim = rbf_kernel(affinity_matrix, gamma=gamma)
    adjacency_matrix = np.zeros_like(rbf_sim)
    for i, row in enumerate(rbf_sim):
        nearest_neighbors_indices = np.argsort(row)[::-1]
        nearest_neighbors_indices = nearest_neighbors_indices[:q]
        for j in nearest_neighbors_indices:
            if i in np.argsort(rbf_sim[j])[::-1][:q]:
                adjacency_matrix[i, j] = 1
                adjacency_matrix[j, i] = 1

    n_components, labels = connected_components(adjacency_matrix)
    return (adjacency_matrix, n_components)

def construct_delta_matrix(A):
    delta_matrix = np.zeros((len(A), len(A)))
    for i in range(len(A)):
        row_sum = np.sum(A[i])
        delta_matrix[i, i] = row_sum
    return delta_matrix

def compute_lrw(L, delta):
    Lrw = np.zeros((len(L), len(L[0])))
    for i in range(len(L)):
        for j in range(len(L[0])):
            if delta[i][i] == 0:
                Lrw[i][j] = 0
            else:
                Lrw[i][j] = L[i][j] / delta[i][i]
    return Lrw

def getY(U):
   # Y = normalize(U, norm='l2', axis=1)
    return U

def get_model(Y, k):
    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans.fit_predict(Y)
    return kmeans


def spectral_cluster(D, gamma, nearest_n_after_rbf, K_clusters, ground_truth):
    (A, n_component) = compute_rbf(D, nearest_n_after_rbf, gamma)
    delta = construct_delta_matrix(A)
    L = delta - A
    Lrw = compute_lrw(L, delta)
    eigval, eigvecs = np.linalg.eig(Lrw)
 #   sorted_indices = np.argsort(eigval)
  #  eigval = eigval[sorted_indices]
   # eigvecs = eigvecs[:, sorted_indices]
    model = get_model(getY(np.real(eigvecs[:, n_component:n_component+K_clusters])), K_clusters)
    precision, recall, f1, conditional_entropy = evaluate(ground_truth, model.labels_)
    return precision, recall, f1, conditional_entropy

In [ ]:
def conditional_entropy(true_labels,predictions):
    n_samples = len(true_labels)
    true_labels = np.array(true_labels)
    predictions = np.array(predictions)
    clusters={}
    cluster_ids = np.unique(predictions)
    for id in cluster_ids:
        cluster = true_labels[predictions == id]
        clusters[id]=cluster
    true_clusters_ids = np.unique(true_labels)
    entropy = 0.0
    for i,cluster in enumerate(clusters.values()):
        cluster = np.array(cluster)
        cluster_entropy = 0.0
        for label in  true_clusters_ids:
            count = np.where(cluster == label)[0].size
            if(count!=0):
                cluster_entropy -= (count/cluster.size)*np.log(count/cluster.size)
        entropy += (cluster.size/n_samples) * cluster_entropy
    return entropy
def percision_recall_f1_score(true_labels,predictions):
  n_samples = len(true_labels)
  true_labels = np.array(true_labels)
  t_labels,true_counts = np.unique(true_labels,return_counts=True)
  predictions = np.array(predictions)
  clusters={}
  cluster_ids = np.unique(predictions)
  for id in cluster_ids:
    cluster = true_labels[predictions == id]
    clusters[id]=cluster
  percision = 0.0
  recall = 0.0
  f1_scores = []
  flag=True
  i=0

  for cluster in clusters.values():
    elements_classes,counts = np.unique(cluster,return_counts=True)
    class_purity = (np.max(counts)/cluster.size)
    percision+=(np.max(counts)/n_samples)
    max_element = elements_classes[np.argmax(counts)]
    class_recall  = np.max(counts)/true_counts[t_labels==max_element][0]
    recall+=class_recall*(cluster.size/n_samples)
    Fi = (2*class_purity*class_recall)/(class_purity+class_recall)
    f1_scores.append(Fi)
  return percision,recall,np.mean(f1_scores)

def evaluate(true_labels, cluster_labels):
    percision,recall,f1 = percision_recall_f1_score(true_labels,cluster_labels)
    conditional_entroy = conditional_entropy(true_labels, cluster_labels)
    return (percision, recall, f1, conditional_entroy)

In [ ]:
def driver_code(address):
    D_train, D_test, ground_truth_train, ground_truth_test = read_data(address)
    D = []
    ground_truth = []
    for avg_or_pca in range(0, 2):
      for test_or_train in range(0, 2): # now we train only
       for pca_dim in [800]:
        if test_or_train == 0:
            D = reduce_dimensions(D_train, avg_or_pca, pca_dim)
            ground_truth = ground_truth_train
        else:
            D = reduce_dimensions(D_test, avg_or_pca, pca_dim)
            ground_truth = ground_truth_test
        for K_clusters in [19]:
            for gamma in [.1, .4, 1, 2]:
                for nearest_n_after_rbf in [20, 100, 400, 800]:
                    spec_precision, spec_recall, spec_f1, spec_conditional_entropy = spectral_cluster(D, gamma, nearest_n_after_rbf,
                                                                                    K_clusters, ground_truth)
                    print(f'{test_or_train}, {avg_or_pca}, {pca_dim}, {K_clusters}, {gamma}, {nearest_n_after_rbf}, '+
                         f'{spec_precision}, {spec_recall}, {spec_f1}, {spec_conditional_entropy}')


driver_code('/content/drive/MyDrive/data/')


# Summary of Spectral Clustering
We've tried so many functions to compute the matrix A, here there are:
``` python
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.csgraph import connected_components

def construct_adjacency_matrix(affinity_matrix, initial_q=15, increment=15):
    num_points = len(affinity_matrix)
    adjacency_matrix = np.zeros((num_points, num_points))
    cosine_sim = cosine_similarity(affinity_matrix)

    # Starting with initial_q
    q = initial_q

    while True:
        # Find the q nearest neighbors for each data point
        nbrs = NearestNeighbors(n_neighbors=q+1, metric='cosine').fit(affinity_matrix)
        distances, indices = nbrs.kneighbors(affinity_matrix)

        # Connect mutually nearest neighbors
        for i in range(num_points):
            for j in indices[i]:
                if i != j and i in indices[j]:
                    adjacency_matrix[i, j] = cosine_sim[i][j]
                    adjacency_matrix[j, i] = cosine_sim[i][j]

        # Check if the graph is connected
        n_components, labels = connected_components(adjacency_matrix)
        if np.max(labels) == 0:
            break  # Graph is connected, exit the loop
        print("need increase")
        print(n_components)
        # Increase q and reset adjacency_matrix
        q += increment
        adjacency_matrix = np.zeros((num_points, num_points))

    return adjacency_matrix
```

``` python
import numpy as np
from scipy.sparse.csgraph import connected_components
def min_positive_number(arr):
    positives = arr[arr > 0]    
    if positives.size == 0:
        return None  
    min_positive = np.min(positives)
    return min_positive
def cos_plus(affinity_matrix, threshold):
    num_points = len(affinity_matrix)
    cosine_sim = cosine_similarity(affinity_matrix)
    min_num_not_equal_zero = min_positive_number(cosine_sim)
    cosine_sim[cosine_sim < threshold] = 0
    cosine_sim[cosine_sim > threshold] = 1
    n_components, labels = connected_components(cosine_sim)
    np.savetxt('j:/row.txt', [np.max(labels), min_num_not_equal_zero])
    return cosine_sim
```
```python
import numpy as np
from scipy.sparse.csgraph import connected_components
from sklearn.metrics.pairwise import cosine_similarity

def cos_plus_plus(affinity_matrix, q):
    cosine_sim = cosine_similarity(affinity_matrix)
    adjacency_matrix = np.zeros_like(cosine_sim)
    for i, row in enumerate(cosine_sim):
        nearest_neighbors_indices = np.argsort(row)[::-1]
        nearest_neighbors_indices = nearest_neighbors_indices[:q]
        for j in nearest_neighbors_indices:
            if i in np.argsort(cosine_sim[j])[::-1][:q]:
                adjacency_matrix[i, j] = 1
                adjacency_matrix[j, i] = 1

    n_components, labels = connected_components(adjacency_matrix)
    np.savetxt('j:/row.txt', [n_components])
    return adjacency_matrix
```
```python
import numpy as np
from scipy.sparse.csgraph import connected_components
from sklearn.metrics.pairwise import cosine_similarity

def cos_plus_plus_plus(affinity_matrix, q):
    cosine_sim = cosine_similarity(affinity_matrix)
    for i, row in enumerate(cosine_sim):
        nearest_neighbors_indices = np.argsort(row)[::-1]
        nearest_neighbors_indices = nearest_neighbors_indices[:q]
        for j in nearest_neighbors_indices:
                cosine_sim[i, j] = 1
                cosine_sim[j, i] = 1

    n_components, labels = connected_components(cosine_sim)
    np.savetxt('j:/row.txt', [n_components])
    return cosine_sim

```
```python
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.csgraph import connected_components
from sklearn.metrics.pairwise import rbf_kernel

def rbf(affinity_matrix, initial_q, increment, gamma):
    num_points = len(affinity_matrix)
    adjacency_matrix = np.zeros((num_points, num_points))
    
    # Starting with initial_q
    q = initial_q

    while True:
        # Compute the RBF kernel matrix
        affinity_matrix_rbf = rbf_kernel(affinity_matrix, gamma=gamma)
        # Find the q nearest neighbors for each data point based on the RBF kernel matrix
        nbrs = NearestNeighbors(n_neighbors=q+1, algorithm='brute', metric='precomputed')
        nbrs.fit(affinity_matrix_rbf)  # Fit the model with the RBF kernel matrix
        distances, indices = nbrs.kneighbors(affinity_matrix_rbf)

        # Connect mutually nearest neighbors
        for i in range(num_points):
            for j in indices[i]:
                if i != j and i in indices[j]:
                    adjacency_matrix[i, j] = affinity_matrix_rbf[i][j]
                    adjacency_matrix[j, i] = affinity_matrix_rbf[i][j]

        # Check if the graph is connected
        n_components, labels = connected_components(adjacency_matrix)
        if np.max(labels) == 0:
            break  # Graph is connected, exit the loop

        print("need increase")
        print(np.max(labels))        
        # Increase q and reset adjacency_matrix
        q += increment
        adjacency_matrix = np.zeros((num_points, num_points))

    return adjacency_matrix
```
```python
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.csgraph import connected_components

def laplacian_kernel(X, sigma=1.0):
    n_samples = X.shape[0]
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            K[i, j] = np.exp(-np.linalg.norm(X[i] - X[j]) / sigma)
    return K

def rbf_laplacian(affinity_matrix, initial_q, increment, sigma):
    num_points = len(affinity_matrix)
    adjacency_matrix = np.zeros((num_points, num_points))
    K = laplacian_kernel(affinity_matrix, sigma)

    # Starting with initial_q
    q = initial_q

    while True:
        # Find the q nearest neighbors for each data point
        nbrs = NearestNeighbors(n_neighbors=q+1, metric='euclidean').fit(affinity_matrix)
        distances, indices = nbrs.kneighbors(affinity_matrix)

        # Connect mutually nearest neighbors
        for i in range(num_points):
            for j in indices[i]:
                if i != j and i in indices[j]:
                    adjacency_matrix[i, j] = K[i][j]
                    adjacency_matrix[j, i] = K[i][j]

        # Check if the graph is connected
        n_components, labels = connected_components(adjacency_matrix)
        if np.max(labels) == 0:
            break  # Graph is connected, exit the loop
        print(q)
        print(np.max(labels))
        print("need_increase")
        # Increase q and reset adjacency_matrix
        q += increment
        adjacency_matrix = np.zeros((num_points, num_points))

    return adjacency_matrix
```
The results of the evaluation were not that good:
```
Cosine simalirity along with 2000 Mutual Nearest Neighbours with L symmetric Average F1 Score: 0.3610305973806317
Weighted Average Recall Score: 0.47286184210526316
Weighted Average Precision Score: 0.29700675805383236

Cosine simalirity along with 400 Mutual Nearest Neighbours with L asymmetric cosine_plus_plus 29 component
Precision : 0.4621895624527028
Weighted Average Recall Score: 0.5748355263157895
Average F1 Score: 0.5153072434351277

Cosine Similarity with 5 Nearest Neighbours with L a cosine_plus_plus_plus
Average F1 Score: 0.4424688804852393
Weighted Average Recall Score: 0.18530701754385964
precision: 0.3791890925050078

cosine_plus_plus with pca 605
Average F1 Score: 0.46028396770241153
Weighted Average Recall Score: 0.5652412280701754
precision: 0.39053510548528986

cosine_plus_plus with pca 605 with test data Average F1 Score: 0.56031938459708
Weighted Average Recall Score: 0.3656798245614035
precision: 0.5147547788582703

```
Untill we used the RBF with nearest neighbours, the results increased a lot:
```
rbf_plus_plus Average F1 Score: 0.7193104198761674
Average Recall Score: 0.7855173956104667
Precision: 0.6779438542874393


rbf_plus_plus with pca 45 component Average F1 Score: 0.7301748607065958
Average Recall Score: 0.7877742818291703
Precision: 0.6999127158336077
```
NOTE: At this stage we were using weighted average in the recall and precision.

Now we'll discuss some plots representing the results of spectral clustering:

1. Different K's  
The bigger the K "number of components", the better the results
![](https://drive.google.com/uc?export=view&id=1Mhk2vwaq6iSBGK2I5aq-LTk9fsNZlASh)

2. Dimensions and Evaluation  
The bigger the dimensions, the better the results.
![](https://drive.google.com/uc?export=view&id=1qUHEgQ-RgKEfFM6YVDWl0pWuBiaa1Bgo)

3. Averaging vs PCA
![](https://drive.google.com/uc?export=view&id=1609qkOnxXRU-pmsy1wI-3v63IHD_4HS5)
Surprisingly, averaging was better on average, although it was less stable.

4. Best Parameters for Spectral Clustering
a.  Gamma  
![](https://drive.google.com/uc?export=view&id=1MbCWEJV_TOq7mohP6hPm4YmRlUhjPJ5D)

In the precision, f1, and conditional entropy the .1 was the best.
So we could say that gamma = .1 is the best one.
![](https://drive.google.com/file/d/1k19zb6vFQB7s7HdZJiwqj8SJNEIQol_K/view?usp=sharing)
b. Nearest n in after RBF  
![](https://drive.google.com/uc?export=view&id=1k19zb6vFQB7s7HdZJiwqj8SJNEIQol_K)
Best were 400.

c. The top Five tuples in the training dataset   
![](https://drive.google.com/uc?export=view&id=1_h7Z8qRMX7Xic8g4WL2mefxmPpmcQzCU)
Best parameters in the training data were gamma = .1, and nearest neighbours was 400. Although there was higher recalls, but they had very low f1's and precisions and higher conditional entropy. And averaging was better than PCA Let's try these parameters on test data and look at the results.  

We optained these resuts:

0.1, 400, 0.21655701754385967, 0.7864983095760237, 0.23657642316391342, 2.3411630133950525

first number is precision, seond is recall, third is f1, fourth is entropy.

​

